In [2]:
using LCMGL
using StaticArrays
import DataStructures: OrderedDict
using RigidBodyDynamics
using ForwardDiff
using Rotations
using DataFrames, DataArrays
using MathProgBase
using Ipopt

In [3]:
include("arms.jl")
include("depth_sensors.jl")
include("depthdata.jl")
include("gradientdescent.jl")
gd = GradientDescent

GradientDescent

In [4]:
squished = DepthData.read_point_cloud(open("data/squishable_squished_xyzrgb.txt"))

PointCloud with origin: [1.37689,0.775602,1.4848] containing 25164 points

In [5]:
DepthData.render_lcmgl(squished)

In [6]:
unsquished = DepthData.read_point_cloud(open("data/squishable_unsquished_xyzrgb.txt"))

PointCloud with origin: [1.38246,0.768824,1.48581] containing 25571 points

In [7]:
DepthData.render_lcmgl(unsquished)

In [8]:
function squishable()
    limbs = OrderedDict{RigidBody, Arms.Limb}()
    
    mechanism = Mechanism(RigidBody{Float64}("world"))
    parent = root_body(mechanism)
    
    joint = Joint("joint1", QuaternionFloating())
    joint_to_parent = Transform3D(Float64, joint.frameBefore, parent.frame)
    body = RigidBody(rand(SpatialInertia{Float64}, CartesianFrame3D("body1")))
    body_to_joint = Transform3D(Float64, body.frame, joint.frameAfter)
    attach!(mechanism, parent, joint, joint_to_parent, body, body_to_joint)
    
    surface_points = Vector{Point3D}()
    skeleton_points = [Point3D(body.frame, @SVector [0, 0, 0])]
    radii = @SVector [0.44/2, 0.40/2, 0.30/2]
    for axis = 1:3
        for i_sign = [-1, 1]
            for j_sign = [-1, 1]
                for theta = [pi/4]
                    v = [0., 0, 0]
                    v[axis] = 1
                    x = [0., 0, 0]
                    i = mod(axis, 3) + 1
                    j = mod(i, 3) + 1
                    a = radii[i] * 1.25
                    b = radii[j] * 1.25
                    x[i] = i_sign * sqrt(a^2 * b^2 / (a^2 * tan(theta)^2 + b^2))
                    x[j] = j_sign * sqrt(b^2 * (1 - b^2 / (a^2 * tan(theta)^2 + b^2)))
                    point = SVector(x...)
                    push!(surface_points, Point3D(body.frame, point))
                end
            end
        end
    end
    limbs[body] = Arms.Limb(surface_points, skeleton_points)
    @show surface_points
    Arms.Model(mechanism, limbs)
end

squishable (generic function with 1 method)

In [9]:
model = squishable()
state = Arms.ModelState(model)
Arms.draw(model, state)

surface_points = RigidBodyDynamics.Point3D[Point3D in "body1": [0.0,-0.15,-0.15],Point3D in "body1": [0.0,-0.15,0.15],Point3D in "body1": [0.0,0.15,-0.15],Point3D in "body1": [0.0,0.15,0.15],Point3D in "body1": [-0.154918,0.0,-0.154918],Point3D in "body1": [0.154918,0.0,-0.154918],Point3D in "body1": [-0.154918,0.0,0.154918],Point3D in "body1": [0.154918,0.0,0.154918],Point3D in "body1": [-0.184985,-0.184985,0.0],Point3D in "body1": [-0.184985,0.184985,0.0],Point3D in "body1": [0.184985,-0.184985,0.0],Point3D in "body1": [0.184985,0.184985,0.0]]


DrakeVisualizer.Visualizer(DrakeVisualizer.Robot(DrakeVisualizer.Link[DrakeVisualizer.Link(DrakeVisualizer.GeometryData[DrakeVisualizer.GeometryData{CoordinateTransformations.IdentityTransformation,GeometryTypes.HomogenousMesh{FixedSizeArrays.Point{3,Float64},GeometryTypes.Face{3,Int64,0},Void,Void,Void,Void,Void}}(HomogenousMesh(
    vertices: 2268xFixedSizeArrays.Point{3,Float64},     faces: 4532xGeometryTypes.Face{3,Int64,0}, )
,CoordinateTransformations.IdentityTransformation(),RGBA{Float64}(1.0,0.0,0.0,0.5))],"skin")]),1,PyLCM.LCM(PyObject <LCM object at 0x7fe53410a2d0>))

In [10]:

function flatten(state::Arms.ModelState)
    data = configuration_vector(state.mechanism_state)
    for limb_deformation in state.limb_deformations
        for d in limb_deformation
            for x in d.v
                push!(data, x)
            end
        end
    end
    data
end

function unflatten(model::Arms.Model, state::Arms.ModelState, data)
    nq = num_positions(state.mechanism_state)
    set_configuration!(state.mechanism_state, data[1:nq])
    
    data_index = nq + 1;
    for (i, (body, limb)) in enumerate(model.limbs)
        limb_deformation = state.limb_deformations[i]
        for (j, deformation) in enumerate(limb_deformation)
            state.limb_deformations[i][j] = RigidBodyDynamics.FreeVector3D(body.frame, @SVector [data[data_index + i] for i in 0:2])
            data_index += 3
        end
    end
    @assert data_index == length(data) + 1
    state
end

unflatten (generic function with 1 method)

In [11]:
point_cloud = unsquished
DepthData.render_lcmgl(point_cloud)
sensed_points = [p.position for p in point_cloud.points]

x_ad = []
state_ad = []

cost = x -> begin
    push!(x_ad, x)
    state = Arms.ModelState(model, eltype(x), eltype(x))
    unflatten(model, state, x)
    push!(state_ad, state)
    Arms.draw(model, state, false)
    skin = Arms.skin(model, state)
    raycast_cost = sum(point -> skin(point)^2, sensed_points)
    deformation_cost = zero(eltype(x))
    for ld in state.limb_deformations
        for d in ld
            deformation_cost += sum(d.v .^2)
        end
    end
    return raycast_cost / length(sensed_points) + 100*deformation_cost
end

# Use autodiff to prepare a function that will evaluate the gradient
# of that cost:
cost_and_gradient = x -> begin
    out = GradientResult(x)
    ForwardDiff.gradient!(out, cost, x)
    ForwardDiff.value(out), ForwardDiff.gradient(out)
end

trials = []


function test_gradient_descent()
    for i = 1:1
        xs = []

        # We'll initialize our estimate with a uniformly random pair
        # of joint angles
        x_estimated = flatten(Arms.ModelState(model))
        x_estimated[5:7] = [.5, 0, .5]

        # Now we use gradient descent to recover the joint angles
        for i = 1:50
            push!(xs, x_estimated)

            # Evaluate the gradient of the point values w.r.t. the
            # joint angles:
            ci, gi = cost_and_gradient(x_estimated)

            if any(isnan, gi)
                @show x_estimated
                error("NaN in gradient")
            end
            
            # Descend the gradient
            step = -ci ./ gi
            step[1:7] = sign(step[1:7]) .* min.(0.01, abs.(gi[1:7]))
            step[8:end] = sign(step[8:end]) .* min.(1e-5, abs.(gi[8:end]))
            x_estimated += step

            # Descend the gradient
#             x_estimated -= 0.0000002 * gi
            
            # TODO: generalize this
            x_estimated[1:4] = x_estimated[1:4] / norm(x_estimated[1:4])
        end
        push!(trials, DataFrame(x=xs))
    end
end

test_gradient_descent (generic function with 1 method)

In [12]:
test_gradient_descent()
state = unflatten(model, Arms.ModelState(model), trials[1][:x][end])
Arms.draw(model, state)

DrakeVisualizer.Visualizer(DrakeVisualizer.Robot(DrakeVisualizer.Link[DrakeVisualizer.Link(DrakeVisualizer.GeometryData[DrakeVisualizer.GeometryData{CoordinateTransformations.IdentityTransformation,GeometryTypes.HomogenousMesh{FixedSizeArrays.Point{3,Float64},GeometryTypes.Face{3,Int64,0},Void,Void,Void,Void,Void}}(HomogenousMesh(
    vertices: 1738xFixedSizeArrays.Point{3,Float64},     faces: 3472xGeometryTypes.Face{3,Int64,0}, )
,CoordinateTransformations.IdentityTransformation(),RGBA{Float64}(1.0,0.0,0.0,0.5))],"skin")]),1,PyLCM.LCM(PyObject <LCM object at 0x7fe53410a300>))

In [15]:
x0 = flatten(Arms.ModelState(model))
num_vars = length(x0)
problem = gd.UnconstrainedOptimization(cost)

solver = IpoptSolver(print_level=1, max_iter=50)
# solver = NLoptSolver(algorithm=:LD_MMA)
m = MathProgBase.NonlinearModel(solver)
MathProgBase.loadproblem!(m, num_vars, 0, [-Inf for i in 1:num_vars], [Inf for i in 1:num_vars], Float64[], Float64[], :Min, problem)
MathProgBase.setwarmstart!(m, x0);

In [16]:
MathProgBase.optimize!(m)
x = MathProgBase.getsolution(m)
state = unflatten(model, Arms.ModelState(model), x)
Arms.draw(model, state)

DrakeVisualizer.Visualizer(DrakeVisualizer.Robot(DrakeVisualizer.Link[DrakeVisualizer.Link(DrakeVisualizer.GeometryData[DrakeVisualizer.GeometryData{CoordinateTransformations.IdentityTransformation,GeometryTypes.HomogenousMesh{FixedSizeArrays.Point{3,Float64},GeometryTypes.Face{3,Int64,0},Void,Void,Void,Void,Void}}(HomogenousMesh(
    vertices: 1600xFixedSizeArrays.Point{3,Float64},     faces: 3196xGeometryTypes.Face{3,Int64,0}, )
,CoordinateTransformations.IdentityTransformation(),RGBA{Float64}(1.0,0.0,0.0,0.5))],"skin")]),1,PyLCM.LCM(PyObject <LCM object at 0x7fe53410a450>))